In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-sep-2021/train.csv
/kaggle/input/tabular-playground-series-sep-2021/test.csv
/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipe=Pipeline([
           ('scaler',StandardScaler()),
           ('imp',SimpleImputer(strategy='mean',missing_values=np.nan)),
])

In [4]:
train_set = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test_set = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
values=test_set['id']
test_set=test_set.drop(['id'],axis=1).copy()

In [5]:
X_train=train_set.drop(['id','claim'],axis=1).copy()
Y_train=train_set['claim'].copy()
Y_train

0         1
1         0
2         1
3         1
4         1
         ..
957914    0
957915    1
957916    0
957917    1
957918    0
Name: claim, Length: 957919, dtype: int64

In [6]:
X_train= pd.DataFrame(columns=X_train.columns, data=num_pipe.fit_transform(X_train))
test_set = pd.DataFrame(columns=test_set.columns, data=num_pipe.transform(test_set))

In [7]:
import catboost
from catboost import CatBoostClassifier

In [8]:
# best_params={'iterations': 15585, 
#     'objective': 'CrossEntropy', 
#     'bootstrap_type': 'Bernoulli', 
#     'od_wait': 1144, 
#     'learning_rate': 0.023575206684596582, 
#     'reg_lambda': 36.30433203563295, 
#     'random_strength': 43.75597655616195, 
#     'depth': 7, 
#     'min_data_in_leaf': 11, 
#     'leaf_estimation_iterations': 1, 
#     'subsample': 0.8227911142845009,
#     'task_type' : 'GPU',
#     'devices' : '0',
#     'verbose' : 0}

In [9]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import roc_curve,auc
# from sklearn.model_selection import StratifiedKFold

# kf=StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
# pred=[]
# for folds,(idx_train,idx_valid) in enumerate(kf.split(X_train,Y_train)):

#   x_train, y_train = X_train.iloc[idx_train], Y_train.iloc[idx_train]
#   x_valid, y_valid = X_train.iloc[idx_valid], Y_train.iloc[idx_valid]


#   model=CatBoostClassifier(**best_params)
#   model.fit(x_train,y_train)

#   prob=model.predict_proba(x_valid)[:,1]
#   fpr, tpr, _ = roc_curve(y_valid, prob)
#   score = auc(fpr, tpr)

#   print(f'Fold:{folds+1} Score:{score}')
#   preds=model.predict_proba(test_set)[:,1]
#   pred.append(preds)

In [10]:
from sklearn.model_selection import cross_val_score,GridSearchCV
params={'iterations':100,'learning_rate':0.5,'objective':'Logloss'}
model=CatBoostClassifier(**params)
model.fit(X_train,Y_train)

0:	learn: 0.6910419	total: 763ms	remaining: 1m 15s
1:	learn: 0.6892538	total: 1.33s	remaining: 1m 5s
2:	learn: 0.6871825	total: 2.04s	remaining: 1m 5s
3:	learn: 0.6854292	total: 2.69s	remaining: 1m 4s
4:	learn: 0.6835987	total: 3.36s	remaining: 1m 3s
5:	learn: 0.6807819	total: 4.38s	remaining: 1m 8s
6:	learn: 0.6778815	total: 5.03s	remaining: 1m 6s
7:	learn: 0.6751540	total: 5.75s	remaining: 1m 6s
8:	learn: 0.6736224	total: 6.46s	remaining: 1m 5s
9:	learn: 0.6707943	total: 7.17s	remaining: 1m 4s
10:	learn: 0.6682158	total: 7.93s	remaining: 1m 4s
11:	learn: 0.6654707	total: 8.62s	remaining: 1m 3s
12:	learn: 0.6629213	total: 9.32s	remaining: 1m 2s
13:	learn: 0.6614995	total: 9.94s	remaining: 1m 1s
14:	learn: 0.6598145	total: 10.6s	remaining: 60s
15:	learn: 0.6573416	total: 11.2s	remaining: 59s
16:	learn: 0.6561221	total: 11.9s	remaining: 58.1s
17:	learn: 0.6540920	total: 12.5s	remaining: 56.9s
18:	learn: 0.6527955	total: 13.2s	remaining: 56.1s
19:	learn: 0.6514117	total: 13.9s	remaining:

In [11]:
pred=model.predict_proba(test_set)

In [12]:
preds=pred[:,1]

In [13]:
sub   = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [14]:
features = [col for col in train_set.columns if col not in ['claim', 'id']]

In [15]:
output2=pd.DataFrame({'id':values,'claim':preds})
output2

,id,claim
0,957919,0.473548
1,957920,0.283231
2,957921,0.396378
3,957922,0.296028
4,957923,0.293263
...,...,...
493469,1451388,0.591560
493470,1451389,0.263996
493471,1451390,0.753757
493472,1451391,0.278399


In [16]:
output2.to_csv('submission.csv', index=False)